<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

#THE CODE

##1. Load and clean the data

In [0]:
### Import
import pandas as pd
import numpy as np

In [3]:
# Now import the data
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-01-31 01:41:16--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2020-01-31 01:41:22 (278 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [4]:
df=pd.read_csv('house-votes-84.data')
df.head()

,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [5]:
# In the current way it's not intuitive to use this dataset.
# I'll rename the columns, taking the data from the "names" file
column_headers = ['party','handicapped-infants','water-project',
                  'budget','physician-fee-freeze','el-salvador-aid',
                  'religious-groups','anti-satellite-ban',
                  'aid-to-contras','mx-missile','immigration',
                  'synfuels', 'education', 'right-to-sue','crime',
                  'duty-free','south-africa']

df=pd.read_csv('house-votes-84.data', header=None, names=column_headers, na_values='?')
# "na_values='?'" converts question marks into NaNs
print(df.shape)
df.head()

(435, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [6]:
# In order to work with the data, we need to convert y/ns into numeric
df=df.replace({'y':1,'n':0})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [7]:
# Let's count the amount of representatives of each party
df['party'].value_counts().sort_index()

democrat      267
republican    168
Name: party, dtype: int64

In [8]:
# Now let's create two different datasets:
# separately for democrats and republicans,
# so we could count them independently
dem=df[df['party']=='democrat']
rep=df[df['party']=='republican']
dem.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [9]:
rep.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


##2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

In [0]:
# import
from scipy.stats import ttest_ind

In [11]:
# First finding the average value for both parties (for later check)
print(dem['handicapped-infants'].mean())
print(rep['handicapped-infants'].mean())

# Now time for the test, NaNs skipped
ttest_ind(dem['handicapped-infants'], rep['handicapped-infants'], nan_policy='omit')
# you're sure that statistically significant in ttest_ind(B, A) B > A if statistic > 0 & pvalue < p

# t goes on to say that scipy always gives the test statistic as signed. 
# This means that given p and t values from a two-tailed test, 
# you would reject the null hypothesis of a greater-than test when p/2 < alpha and t > 0, 
# and of a less-than test when p/2 < alpha and t < 0.

0.6046511627906976
0.18787878787878787


Ttest_indResult(statistic=9.205264294809222, pvalue=1.613440327937243e-18)

##3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01

In [12]:
# Same process for democrats
print(dem['physician-fee-freeze'].mean())
print(rep['physician-fee-freeze'].mean())
ttest_ind(dem['physician-fee-freeze'], rep['physician-fee-freeze'], nan_policy='omit')

0.05405405405405406
0.9878787878787879


Ttest_indResult(statistic=-49.36708157301406, pvalue=1.994262314074344e-177)

##4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

In [13]:
# 
print(ttest_ind(rep['handicapped-infants'],dem['handicapped-infants'],nan_policy='omit'))
# print(ttest_ind(rep['water-project'],dem['water-project'],nan_policy='omit'))
print(ttest_ind(rep['budget'],dem['budget'],nan_policy='omit'))
print(ttest_ind(rep['physician-fee-freeze'],dem['physician-fee-freeze'],nan_policy='omit'))
print(ttest_ind(rep['el-salvador-aid'],dem['el-salvador-aid'],nan_policy='omit'))
print(ttest_ind(rep['religious-groups'],dem['religious-groups'],nan_policy='omit'))
print(ttest_ind(rep['anti-satellite-ban'],dem['anti-satellite-ban'],nan_policy='omit'))
print(ttest_ind(rep['crime'],dem['crime'],nan_policy='omit'))
print(ttest_ind(rep['south-africa'],dem['south-africa'],nan_policy='omit'))

Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)
Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)
Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)
Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)
Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)
Ttest_indResult(statistic=16.342085656197696, pvalue=9.952342705606092e-47)
Ttest_indResult(statistic=-6.849454815841208, pvalue=3.652674361672226e-11)
